# Sensitivity results in Taylor diagram
- This script is used to visualize sensitivity results in Taylor diagram;
- Simulations: BASE, SENS;

In [37]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cftime
import string
import matplotlib.dates as mdates
import geocat.viz as gv
from matplotlib.lines import Line2D

In [38]:
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_sensitivity/taylor_diagram/'
result = pd.read_csv('result.csv')
result

,Variable,Index,Type,sdev,coef,site
0,SWup,0,morp,1.100191,0.997359,AU-Preston
1,LWup,0,morp,1.177265,0.974246,AU-Preston
2,Qh,0,morp,0.748667,0.927966,AU-Preston
3,Qle,0,morp,0.536699,0.436427,AU-Preston
4,Qtau,0,morp,0.233464,0.870784,AU-Preston
...,...,...,...,...,...,...
5935,SWup,8,temp,0.965178,0.997931,US-WestPhoenix
5936,LWup,8,temp,0.982153,0.996330,US-WestPhoenix
5937,Qh,8,temp,0.829161,0.926568,US-WestPhoenix
5938,Qle,8,temp,0.249344,0.536711,US-WestPhoenix


In [51]:
labelfont = 14
linewidth = 1
labelcolor = '#6b6b6b'
padding = 8
pad = 1
xwidth = 1
length = 1
maintitles = ['SWup', 'LWup', 'Qh', 'Qle', 'Qtau']
psites = ["AU-Preston","US-Baltimore","US-WestPhoenix"]
#psites = ["AU-Preston","US-Minneapolis1","US-WestPhoenix"]
lc0 = '#083550'
lc1 = '#006bac'
lc2 = '#3da4e3'
lc3 = '#6c64d4'
lc4 = '#1a8865'
lc5 = '#cb682c'
colors = [lc1, lc2, lc3, lc4, lc5]

case_index = 'Index'
site_index = 'site'
var_index = 'Variable'
marker = ['*', 'o', 'D']
fig = plt.figure(figsize=(13, 9))
data_list = []
subset_label = ['morp', 'radi', 'ther', 'temp']
subset = ['Morphological', 'Radiative', 'Thermal', 'Indoor']
index_length = [65, 257, 65, 9]
marker_list = ['☆', '○', '◇', '1']
#site_list = ["AU-Pre","US-Mi1","US-Wes", 'Baseline']
site_list = ["AU-Pre","US-Bal","US-Wes", 'BASE']
var_list = ['$SW_{up}$', '$LW_{up}$', '$Q_{h}$', '$Q_{le}$', '$Q_{tau}$'] 

<Figure size 1300x900 with 0 Axes>

In [52]:
for penal, sublable in enumerate(subset_label):
    results_df = result[result['Type']==sublable]
    sub_index = index_length[penal]
    subset_var = subset[penal]
    da = gv.TaylorDiagram(refstd=1, fig=fig, rect=221 + penal, label='REF') # create sub-plot
    list = f"({chr(97 + penal)})" 
    da.add_title(list + ' ' + subset[penal], y_loc = 1.05, fontsize=labelfont)
    da.set_fontsizes_and_pad(ticklabel_fontsize = labelfont, axislabel_fontsize=labelfont, axislabel_pad = 1)
    da.add_std_grid(np.array([0.5, 1.5]), color=labelcolor, linewidth=xwidth)
    da.add_corr_grid(np.array([0.6, 0.9]), color=labelcolor,linewidth=xwidth)
    da.add_contours(levels=np.arange(0, 1.5, 0.25), colors=labelcolor, linewidths=xwidth, **{'linestyles': '--'})  
    for s, site_index in enumerate(psites):
        data_sel = results_df[results_df['site']==site_index]
        if s==0:
             factor = 2
        else:
             factor = 1
        for case in range(2):
            if case==0:
                data = data_sel[data_sel['Index']<sub_index-1]
                markersize = 15 * factor
                markerline =0.25
                annotate_on=False
            else:
                data = data_sel[data_sel['Index']==sub_index-1]
                markersize = 60 * factor
                markerline = 1.5
                annotate_on= True
            data['Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
            #data.loc[:, 'Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
            std_pivot_df = data.pivot(index=var_index, columns=case_index, values='sdev')
            cof_pivot_df = data.pivot(index=var_index, columns=case_index, values='coef')
            stddev = std_pivot_df.to_numpy()
            corrcoef = cof_pivot_df.to_numpy()
            
            # Add models case by case
            for j in range(stddev.shape[0]): # stddev.shape[0]=5
                for m in range(stddev.shape[0]):
                    da.add_model_set(stddev[j],
                         corrcoef[j],
                         xytext=(-13.5, 24.5), # marker label location, in pixels
                         fontsize=labelfont-2,
                         color=colors[j], # label color
                         annotate_on=annotate_on,
                         #label='Baseline',
                         marker=marker[s], 
                         facecolors='none',
                         s=markersize,
                         linewidth=markerline)
                #da.add_model_name(maintitles, fontsize=16)




for p, marker in enumerate(marker_list):
    for q, var in enumerate(var_list):
        if p<3:
            text = marker + '  ' + var + ' at ' + site_list[p]
        else:
            text = marker + '  ' + var + ' ' + site_list[p]
        fig.text(0.85, 0.75 - (0.15 * p + 0.025 * q) , text, fontsize=12, color=colors[q])

fig.subplots_adjust(left=0.05, right=0.8, bottom=0.035, top=0.95, wspace=0.5, hspace=0.3)
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_sensitivity/taylor_diagram/'
fig.savefig(output_dir + 'taylor_sens.png', dpi=300)    
fig.savefig(output_dir + 'taylor_sens.pdf', dpi=600) 

/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Variable'] = pd.Categorical(data['Variable'], categories=maintitles, ordered=True)
/tmp/5310106.1.parallel.q/ipykernel_18019/488020656.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of

# Description

In [41]:
index_table = pd.read_csv('../create_lcz_input_for_sensivity/index_list.csv')
index_table

,Unnamed: 0,Index,label_longname,Type
0,0,0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
1,1,1,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
2,2,2,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
3,3,3,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
4,4,4,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
...,...,...,...,...
387,387,3,T_BUILDING_MIN_1.05,temp
388,388,4,T_BUILDING_MIN_0.95,temp
389,389,5,T_BUILDING_MIN_0.9,temp
390,390,6,T_BUILDING_MIN_0.85,temp


In [42]:
merged = pd.merge(result, index_table, on=['Index', 'Type'], how='left')
merged

,Variable,Index,Type,sdev,coef,site,Unnamed: 0,label_longname
0,SWup,0,morp,1.100191,0.997359,AU-Preston,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
1,LWup,0,morp,1.177265,0.974246,AU-Preston,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
2,Qh,0,morp,0.748667,0.927966,AU-Preston,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
3,Qle,0,morp,0.536699,0.436427,AU-Preston,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
4,Qtau,0,morp,0.233464,0.870784,AU-Preston,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
...,...,...,...,...,...,...,...,...
5935,SWup,8,temp,0.965178,0.997931,US-WestPhoenix,NaN,NaN
5936,LWup,8,temp,0.982153,0.996330,US-WestPhoenix,NaN,NaN
5937,Qh,8,temp,0.829161,0.926568,US-WestPhoenix,NaN,NaN
5938,Qle,8,temp,0.249344,0.536711,US-WestPhoenix,NaN,NaN


In [123]:
merged.to_csv('merged.csv')

In [44]:
morp =merged[merged['Type']=='morp']
morp_qtau = morp[morp['Variable']=='Qtau']
#morp_qtau_mi1 = morp_qtau[morp_qtau['site']=="US-Minneapolis1"]
morp_qtau_bal = morp_qtau[morp_qtau['site']=="US-Baltimore"]
morp_qtau_pre = morp_qtau[morp_qtau['site']=="AU-Preston"]
morp_qtau_wes = morp_qtau[morp_qtau['site']=="US-WestPhoenix"]

,Variable,Index,Type,sdev,coef,site,Unnamed: 0,label_longname
329,Qtau,0,morp,0.138587,0.890085,US-Baltimore,0.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
334,Qtau,1,morp,0.139091,0.891965,US-Baltimore,1.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
339,Qtau,2,morp,0.251912,0.877756,US-Baltimore,2.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
344,Qtau,3,morp,0.253287,0.882500,US-Baltimore,3.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
349,Qtau,4,morp,0.138711,0.891467,US-Baltimore,4.0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...
...,...,...,...,...,...,...,...,...
629,Qtau,60,morp,0.126286,0.896882,US-Baltimore,60.0,CANYON_HWR_0.8_HT_ROOF_0.8_THICK_ROOF_0.8_THIC...
634,Qtau,61,morp,0.126698,0.899030,US-Baltimore,61.0,CANYON_HWR_0.8_HT_ROOF_0.8_THICK_ROOF_0.8_THIC...
639,Qtau,62,morp,0.222120,0.889952,US-Baltimore,62.0,CANYON_HWR_0.8_HT_ROOF_0.8_THICK_ROOF_0.8_THIC...
644,Qtau,63,morp,0.223554,0.895318,US-Baltimore,63.0,CANYON_HWR_0.8_HT_ROOF_0.8_THICK_ROOF_0.8_THIC...


In [98]:
morp_qh = morp[morp['Variable']=='Qh']
morp_qh_pre = morp_qh[morp_qh['site']=="AU-Preston"]
print(morp_qh_pre['sdev'].min(), morp_qh_pre['sdev'].max())

0.7012379754071302 0.9990334974756396


In [99]:
ther =merged[merged['Type']=='ther']
ther_qh = ther[ther['Variable']=='Qh']
ther_qh_pre = ther_qh[ther_qh['site']=="AU-Preston"]
print(ther_qh_pre['sdev'].min(), ther_qh_pre['sdev'].max())

0.7792112310040172 0.9650390598221188


In [76]:
print(morp_qtau_wes['sdev'].min(), morp_qtau_wes['sdev'].max())

0.2525969789911507 0.5242148835160253


In [79]:
min_row = morp_qtau_wes.loc[morp_qtau_wes['sdev'].idxmin()]
min_row['label_longname']

'CANYON_HWR_1.2_HT_ROOF_0.8_THICK_ROOF_0.8_THICK_WALL_0.8_WTLUNIT_ROOF_1.2_WTROAD_PERV_1.2'

In [80]:
max_row = morp_qtau_wes.loc[morp_qtau_wes['sdev'].idxmax()]
max_row['label_longname']

'CANYON_HWR_0.8_HT_ROOF_1.2_THICK_ROOF_1.2_THICK_WALL_1.2_WTLUNIT_ROOF_0.8_WTROAD_PERV_0.8'

In [47]:
print(np.round(morp_qtau_pre['coef'].mean(),1), np.round(morp_qtau_pre['coef'].std(),2))
#print(morp_qtau_mi1['coef'].mean(), morp_qtau_mi1['coef'].std())
print(np.round(morp_qtau_bal['coef'].mean(),1), np.round(morp_qtau_bal['coef'].std(),2))
print(np.round(morp_qtau_wes['coef'].mean(),1), np.round(morp_qtau_wes['coef'].std(),2))

0.9 0.01
0.9 0.01
0.8 0.01


In [83]:
radi =merged[merged['Type']=='radi']
radi_swup = radi[radi['Variable']=='SWup']
radi_swup_mi1 = radi_swup[radi_swup['site']=="US-Minneapolis1"]
radi_swup_pre = radi_swup[radi_swup['site']=="AU-Preston"]
radi_swup_wes = radi_swup[radi_swup['site']=="US-WestPhoenix"]        

In [93]:
radi_qh = radi[radi['Variable']=='Qh']
radi_qh_mi1 = radi_qh[radi_qh['site']=="US-Minneapolis1"]
radi_qh_pre = radi_qh[radi_qh['site']=="AU-Preston"]
radi_qh_wes = radi_qh[radi_qh['site']=="US-WestPhoenix"]  

In [87]:
print(radi_swup_pre['sdev'].min(), radi_swup_pre['sdev'].max())

0.8750899375039309 1.335486317809585


In [89]:
min_row = radi_swup_pre.loc[radi_swup_pre['sdev'].idxmin()]
min_row['label_longname']

'ALB_IMPROAD_DIF_0.8_ALB_PERROAD_DIF_0.8_ALB_ROOF_DIF_0.8_ALB_WALL_DIF_0.8_EM_IMPROAD_1.2_EM_PERROAD_1.2_EM_ROOF_1.2_EM_WALL_1.2'

In [94]:
min_row = radi_qh_pre.loc[radi_qh_pre['sdev'].idxmin()]
min_row['label_longname']

'ALB_IMPROAD_DIF_1.2_ALB_PERROAD_DIF_1.2_ALB_ROOF_DIF_1.2_ALB_WALL_DIF_1.2_EM_IMPROAD_1.2_EM_PERROAD_1.2_EM_ROOF_1.2_EM_WALL_1.2'

In [90]:
max_row = radi_swup_pre.loc[radi_swup_pre['sdev'].idxmax()]
max_row['label_longname']

'ALB_IMPROAD_DIF_1.2_ALB_PERROAD_DIF_1.2_ALB_ROOF_DIF_1.2_ALB_WALL_DIF_1.2_EM_IMPROAD_1.2_EM_PERROAD_1.2_EM_ROOF_1.2_EM_WALL_1.2'